# Biodegradability

In this notebook, we will demonstrate how to **combine relational learning with a Bayesian hyperparameter optimization**.

Before we begin, we should note that getML's relational learning algorithms do not require hyperparameter optimization. This sets them apart from some other machine learning algorithms, most notably deep neural networks. If you are working with deep neural networks, you will have to spend a lot of time on optimizing your hyperparameters. getML's algorithms are different: **You will usually already get decent results with the default hyperparameters.** 

Therefore, you should conduct a hyperparameter optimization when your goal is to **aggressively optimize predictive performance**.

https://archive.ics.uci.edu/ml/datasets/QSAR+biodegradation

* Mansouri, K., Ringsted, T., Ballabio, D., Todeschini, R., Consonni, V. (2013). Quantitative Structure - Activity Relationship models for ready biodegradability of chemicals. Journal of Chemical Information and Modeling, 53, 867-878

In [19]:
import numpy
import random

import getml

getml.engine.set_project('biodegradability')

Creating new project 'biodegradability'


Establish a connection to the remote MariaDB database.

In [20]:
getml.database.connect_mysql(
    host="relational.fit.cvut.cz",
    port=3306,
    dbname="Biodegradability",
    user="guest",
    password="relational",
    time_formats=['%Y/%m/%d']
)

Load all required tables and upload them into the getML engine.

In [21]:
atom = getml.data.DataFrame.from_db('atom','atom')
bond = getml.data.DataFrame.from_db('bond','bond')
gmember = getml.data.DataFrame.from_db('gmember','gmember')
group = getml.data.DataFrame.from_db('group','group')
molecule = getml.data.DataFrame.from_db('molecule','molecule')

Annotate the loaded data.

In [22]:
molecule.set_role('molecule_id', getml.data.roles.join_key)
molecule.set_role('mweight', getml.data.roles.numerical)
molecule.set_role('activity', getml.data.roles.numerical)
molecule.set_role('logp', getml.data.roles.target)

atom.set_role('atom_id', getml.data.roles.join_key)
atom.set_role('molecule_id', getml.data.roles.join_key)
atom.set_role('type', getml.data.roles.categorical)

bond.set_role('atom_id', getml.data.roles.join_key)
bond.set_role('atom_id2', getml.data.roles.join_key)
bond.set_role('type', getml.data.roles.categorical)

gmember.set_role('atom_id', getml.data.roles.join_key)
gmember.set_role('group_id', getml.data.roles.join_key)

group.set_role('group_id', getml.data.roles.join_key)
group.set_role('type', getml.data.roles.categorical)

Split the molecule data table into training, validation, and test set.

In [23]:
percentage_training = 0.5
percentage_validation = 0.25
# Add a column containing a shuffled version of the row numbers used to split the data at random.
molecule.add(numpy.array(random.sample(range(0, molecule.shape[0]), k=molecule.shape[0])), 'index')

molecule_training = molecule.where('molecule_training', 
                                   molecule['index'] < molecule.shape[0]*percentage_training)
molecule_validation = molecule.where('molecule_validation',
                                     (molecule['index'] > molecule.shape[0]*percentage_training) &
                                   (molecule['index'] < molecule.shape[0]*(percentage_training+percentage_validation)))
molecule_testing = molecule.where('molecule_testing', 
                                   molecule['index'] > molecule.shape[0]*(percentage_training+percentage_validation))

In [24]:
atom.save()
bond.save()
gmember.save()
group.save()
molecule_training.save()
molecule_testing.save()
molecule_validation.save()

| molecule_id | logp   | mweight   | activity  | index        | 
| join key    | target | numerical | numerical | unused float | 
---------------------------------------------------------------
| i100_02_7i  | 1.91   | 139.11    | 4.53367   | 231          | 
| i100_41_4i  | 3.03   | 106.167   | 5.04986   | 177          | 
| i101_61_1i  | 4.37   | 254.375   | 7.82244   | 175          | 
| i101_68_8i  | 5.22   | 250.256   | 6.04025   | 167          | 
| i101_77_9i  | 2.18   | 198.268   | 4.56435   | 178          | 
| i106_42_3i  | 3.09   | 106.167   | 6.04025   | 223          | 
| i106_88_7i  | 0.86   | 72.1062   | 6.04025   | 208          | 
| i106_99_0i  | 2.03   | 54.0914   | 6.04025   | 197          | 
| i107_05_1i  | 1.93   | 76.5255   | 6.04025   | 190          | 
| i108_38_3i  | 3.09   | 106.167   | 6.04025   | 240          | 
| i108_95_2i  | 1.51   | 94.1124   | 3.80666   | 224          | 
| i110_86_1i  | 0.8    | 79.1015   | 4.56435   | 220          | 
| i111_42_2i  | -1.71  | 1

Create the data model.

In [25]:
ph_molecule = molecule.to_placeholder()
ph_atom = atom.to_placeholder()
ph_bond = bond.to_placeholder()
ph_gmember = gmember.to_placeholder()
ph_group = group.to_placeholder()

ph_molecule.join(
    ph_atom,
    join_key = 'molecule_id'
)
ph_atom.join(
    ph_bond,
    join_key = 'atom_id'
)
ph_atom.join(
    ph_bond,
    join_key = 'atom_id',
    other_join_key = 'atom_id2'
)
ph_atom.join(
    ph_gmember,
    join_key = 'atom_id'
)
ph_gmember.join(
    ph_group,
    join_key = 'group_id'
)

Construct the feature engineerer, feature selector, and predictor.

In [26]:
feature_selector = getml.predictors.XGBoostRegressor(
    booster='gblinear',
    n_estimators=60,
    n_jobs=6,
    max_depth=7
)

predictor = getml.predictors.XGBoostRegressor(
    booster='gblinear',
    n_estimators=60,
    n_jobs=6,
    max_depth=7
)

## -------------------------------------------------------------------

## Construct the base model.
base_model=getml.models.RelboostModel(
    name='base',
    population=ph_molecule,
    peripheral=[ph_atom, ph_gmember, ph_group, ph_bond],
    loss_function=getml.models.loss_functions.SquareLoss(),
    num_features=50,
    num_subfeatures=10,
    feature_selector=feature_selector,
    predictor=predictor,
    num_threads=3
).send()

Perform the initial training

In [27]:
base_model = base_model.fit(
    population_table=molecule_training,
    peripheral_tables=[atom, gmember, group, bond]
)

Loaded data. Features are now being trained...
Trained model.
Time taken: 0h:0m:20.236894



In [28]:
base_model.score(population_table=molecule_validation,
                peripheral_tables=[atom, gmember, group, bond])

{'mae': [0.6256818797420941],
 'rmse': [0.9089659525390064],
 'rsquared': [0.8510611660062808]}

Perform a Gaussian hyperparameter optimization to find the best possible set of hyperparameters.

In [29]:
## Build a parameter space to search in
param_space = dict()

param_space["max_depth"] = [1, 10]
param_space["min_num_samples"] = [10, 100]
param_space["num_features"] = [10, 100]
param_space["share_selected_features"] = [0.3, 1.0]
param_space["shrinkage"] = [0.01, 0.4]

# Any hyperparameters that relate to the predictor
# are preceded by "predictor_".
param_space["predictor_n_estimators"] = [40, 140]
param_space["predictor_max_depth"] = [3, 15]
param_space["predictor_reg_lambda"] = [0.0, 10.0]

## -------------------------------------------------------------------

## Start the hyperparameter optimization.
gauss_search = getml.hyperopt.GaussianHyperparameterSearch(
    model=base_model,
    param_space=param_space,
    n_iter=120,
    ratio_iter=0.8
)

gauss_search.fit(
  population_table_training=molecule_training,
  population_table_validation=molecule_validation,
  peripheral_tables=[atom, gmember, group, bond]
)


Launched hyperparameter optimization...


Extract the model, which resulted in the lowest RMSE.

In [33]:
res_scores = gauss_search.get_scores()

best_model_name = ''
best_score = numpy.finfo(numpy.float).max

for kkey in res_scores:
    if res_scores[kkey]['rmse'][0] < best_score:
        best_score = res_scores[kkey]['rmse'][0]
        best_model_name = kkey

print(best_model_name)
print(best_score)

2020-03-24T17-07-50-hyperopt-gaussian-relboost-120
0.8057118798438392


Very well, you might say. But how do we know that we didn't just massively overfit to the validation set?

Good point. That's why we still have the testing set.

In [34]:
base_model_testing = base_model.copy('base_model_testing')

best_model = getml.models.load_model(best_model_name)
best_model_testing = best_model.copy('best_model_testing')

scores = base_model_testing.score(
    population_table=molecule_testing,
    peripheral_tables=[atom, gmember, group, bond]
)

print("Test scores base model:")
print(scores)

scores = best_model_testing.score(
    population_table=molecule_testing,
    peripheral_tables=[atom, gmember, group, bond]
)

print("Test scores best model:")
print(scores)

Test scores base model
{'mae': [0.555690715106917], 'rmse': [0.7698695019927322], 'rsquared': [0.8457635624780674]}
Test scores best model
{'mae': [0.5399810405241117], 'rmse': [0.7067350269328148], 'rsquared': [0.8661407763769823]}


As we can see, the evaluation on the testing set still outperforms our base model. Again - not by much. This goes back to our initial statement: Unlike deep neural networks **relational boosting does not require extensive hyperparameter optimization**. You will usually already get decent results with the default parameters or by manually setting some parameters on your own.

Hyperparameter optimizations like this are only recommended if your goal is to really optimize your predictive performance.